In [32]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json

In [36]:
# We need to fetch current USD-RUB rate
pr = requests.get("https://www.cbr-xml-daily.ru/daily_json.js")
p = json.loads(pr.content)
usd = p["Valute"]["USD"]["Value"]
eur = p["Valute"]["EUR"]["Value"]
uah = p["Valute"]["UAH"]["Value"]
kzt = p["Valute"]["KZT"]["Value"]

In [37]:
divisions=["backend", "frontend", "apps", "software", "testing", "administration", "design", "management",\
           "marketing", "analytics", "sale", "content", "support", "hr", "office", "telecom", "other", "security"]
prices_avg = {}
prices_max = {}
headers = {
    "User-Agent": "Mozilla/5.0 \
           (Macintosh; Intel Mac OS X 10_10_1) \
           AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/39.0.2171.95 Safari/537.36"
}

In [38]:
for i in tqdm(divisions, desc="parsing..."):
    page = 0
    prices_avg[i] = []
    prices[i] = []
    while True:
        if page == 0:
            req = requests.get("https://career.habr.com/vacancies?divisions[]=" + i + "&type=all", headers=headers)
        else:
            req = requests.get("https://career.habr.com/vacancies?divisions[]=" + i + "&type=all&page=" + str(page),\
                              headers=headers)
        if not req.ok:
            break
        sp = BeautifulSoup(req.content, "html.parser")
        vc = sp.find_all("div", "vacancy-card")
        if vc == []:
            break
        for v in vc:
            price_raw = v.find("div", "vacancy-card__salary").find("div", "basic-salary").text
            if price_raw.strip() == "":
                continue
            price_s = price_raw.split()
            nums = []
            # We need to deal with spaces between numbers
            for p in range(1, len(price_s)):
                if price_s[p].isdigit() and price_s[p-1].isdigit():
                    nums[len(nums)-1] += price_s[p]
                elif price_s[p].isdigit():
                    nums.append(price_s[p])
            # Now we need to check the valute
            v_sym = price_s[len(price_s) - 1]
            mul = 1
            if not v_sym.isdigit():
                if v_sym == '₽':
                    pass
                elif v_sym == '$':
                    mul = usd
                elif v_sym == '€':
                    mul = eur
                elif v_sym == '₴':
                    mul = uah
                elif v_sym == '₸':
                    mul = kzt
            price_d = list(map(lambda x: int(x)*mul, nums))
            price = sum(price_d)/len(price_d)
            prices_avg[i].append(price)
            prices[i].append(price_d[len(price_d)-1])
        page += 1

parsing...: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 18/18 [05:41<00:00, 18.99s/it]


In [40]:
# Calculaing averages
p_avg = {}
p_avgmax = ("", 0)
p_max = {}
p_max_n = ("", 0)
for i in prices_avg:
    p_avg[i] = sum(prices_avg[i])/len(prices_avg[i])
    if p_avg[i] > p_avgmax[1]:
        p_avgmax = (i, p_avg[i])
for i in prices:
    p_max[i] = max(prices[i])
    if p_max[i] > p_max_n[1]:
        p_max_n = (i, p_max[i])

In [43]:
print("======Average salary======")
for k in p_avg:
    print(k, ": ", int(p_avg[k]), '₽', sep="")
    
print("=======Max salaries=======")

for k in p_max:
    print(k, ": ", int(p_max[k]), '₽', sep="")

print("==========================")
print("The highest average salary is in the ", p_avgmax[0], " division (", int(p_avgmax[1]), ")", sep="")
print("The absolute highest salary is in the ", p_max_n[0], " division (", int(p_max_n[1]), ")", sep="")

======Average salary======
backend: 205283₽
frontend: 178449₽
apps: 210247₽
software: 226479₽
testing: 150429₽
administration: 190334₽
design: 146922₽
management: 187976₽
marketing: 137093₽
analytics: 176217₽
sale: 168885₽
content: 71579₽
support: 91021₽
hr: 108108₽
office: 40000₽
telecom: 175000₽
other: 183171₽
security: 230000₽
=======Max salaries=======
backend: 3633209₽
frontend: 1089963₽
apps: 500000₽
software: 8532650₽
testing: 350000₽
administration: 726641₽
design: 300000₽
management: 581313₽
marketing: 252915₽
analytics: 421525₽
sale: 300000₽
content: 150000₽
support: 260000₽
hr: 252915₽
office: 40000₽
telecom: 250000₽
other: 337220₽
security: 350000₽
The highest average salary is in the security division (230000)
The absolute highest salary is in the software division (8532650)
